#Assignment Set 4

by **Amir Mehrpanah** for Deep Learing Course at Data Science Center, SBU

##Preparing Data

In [81]:
transformations = transforms.Compose([transforms.ToTensor(),
                                      transforms.Normalize([.5],[.5])])

# download data and apply transformations
dataset = torchvision.datasets.FashionMNIST(root = 'data/',
                                              download = True,train = True,
                                              transform = transformations)

# split data into two sets of 70/30 ratio
train_set, test_set = torch.utils.data.random_split(dataset, [42000, 18000])

##Implementing The Fit Function

In [75]:
device = torch.device("cuda:0")

def fit(model, train_data,loss_fn,optimizer,epochs,batch_size):
    # use DataLoader
    train_data_loader = DataLoader(train_data,batch_size=batch_size)

    model.train()
    for epoch in range(epochs):
        true_pred = 0
        for batch_idx, (train_imgs, train_labels) in enumerate(train_data_loader):
            train_imgs = train_imgs.to(device)
            train_labels = train_labels.to(device)

            optimizer.zero_grad()
            soft_y_pred = model(train_imgs)
            loss = loss_fn(soft_y_pred, train_labels)
            loss.backward()
            optimizer.step()
            y_pred = torch.max(soft_y_pred.data, 1)[1] 
            true_pred += (y_pred == train_labels).sum()
            
            if batch_idx % 500 == 0:
                print('Epoch : {} \tLoss: {:.6f}\t Accuracy:{:.3f}%'.format(epoch,
                                                                            loss.item(),
                                                                            float(true_pred*100) / float(batch_size*(batch_idx+1))))

def evaluate(model,batch_size,test_data):
    test_data_loader = DataLoader(test_data,batch_size=batch_size)
    true_pred = 0 
    for test_imgs, test_labels in test_data_loader:
        test_imgs = test_imgs.to(device)
        test_labels = test_labels.to(device)

        soft_y_pred = model(test_imgs)
        y_pred = torch.max(soft_y_pred,1)[1]
        true_pred += (y_pred == test_labels).sum()
    print("Test accuracy:{:.3f}% ".format(float(100*true_pred) / (len(test_data_loader)*batch_size)))

##NN.Sequential

In [87]:
model = nn.Sequential(nn.Conv2d(1, 64,3),
                      nn.ReLU(),

                      nn.MaxPool2d(2),
                      
                      nn.Conv2d(64,64,3),
                      nn.ReLU(),
                      
                      nn.Flatten(),
                      nn.Sigmoid(),

                      nn.Linear(7744,64),
                      nn.ReLU(),
                      
                      nn.Linear(64,32),
                      nn.ReLU6(),

                      nn.Linear(32,10),                      
                      nn.Softmax())

model = model.to(device)
loss_fn = nn.CrossEntropyLoss()
learning_rate = 1e-4
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
fit(model,
    train_data=train_set,
    loss_fn=loss_fn,
    optimizer=optimizer,
    epochs=15,
    batch_size=64)

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:117: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


Epoch : 0 	Loss: 2.303888	 Accuracy:6.250%
Epoch : 0 	Loss: 2.087528	 Accuracy:31.269%
Epoch : 1 	Loss: 1.971548	 Accuracy:60.938%
Epoch : 1 	Loss: 1.892798	 Accuracy:66.648%
Epoch : 2 	Loss: 1.807894	 Accuracy:70.312%
Epoch : 2 	Loss: 1.829028	 Accuracy:74.152%
Epoch : 3 	Loss: 1.767563	 Accuracy:71.875%
Epoch : 3 	Loss: 1.793944	 Accuracy:76.138%
Epoch : 4 	Loss: 1.746357	 Accuracy:75.000%
Epoch : 4 	Loss: 1.773741	 Accuracy:77.330%
Epoch : 5 	Loss: 1.731200	 Accuracy:75.000%
Epoch : 5 	Loss: 1.757480	 Accuracy:78.290%
Epoch : 6 	Loss: 1.722847	 Accuracy:75.000%
Epoch : 6 	Loss: 1.744038	 Accuracy:79.232%
Epoch : 7 	Loss: 1.718830	 Accuracy:75.000%
Epoch : 7 	Loss: 1.735199	 Accuracy:79.993%
Epoch : 8 	Loss: 1.717300	 Accuracy:75.000%
Epoch : 8 	Loss: 1.728971	 Accuracy:80.548%
Epoch : 9 	Loss: 1.715093	 Accuracy:75.000%
Epoch : 9 	Loss: 1.723411	 Accuracy:81.004%
Epoch : 10 	Loss: 1.713673	 Accuracy:75.000%
Epoch : 10 	Loss: 1.718517	 Accuracy:81.462%
Epoch : 11 	Loss: 1.710621	 Acc

In [88]:
evaluate(model,batch_size=64,test_data=test_set)

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:117: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


Test accuracy:81.998% 


##NN.Module

In [78]:
class Model(nn.Module):
    def __init__(self):
        super(Model,self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1,out_channels=32,kernel_size=3)
        self.conv2 = nn.Conv2d(in_channels=32,out_channels=64,kernel_size=5)    
        self.lin1 = nn.Linear(in_features=64*3*3,out_features=100)    
        self.lin2 = nn.Linear(in_features=100,out_features=64)    
        self.lin3 = nn.Linear(in_features=64,out_features=32)    
        self.lin4 = nn.Linear(in_features=32,out_features=10)  
        
    def forward(self,x): 
        # layer 1   
        x = self.conv1(x)
        x = torch.relu(x)
        # layer 2
        x = torch.max_pool2d(x,kernel_size=3,stride=2)
        x = self.conv2(x)
        x = torch.sigmoid(x)
        # layer 3
        x = torch.max_pool2d(x,kernel_size=3,stride=2)
        x = x.reshape(-1,64*3*3)
        x = self.lin1(x)
        x = torch.relu(x)
        # layer 4
        x = self.lin2(x)
        x = torch.relu(x)    
        # layer 5
        x = self.lin3(x)
        x = torch.relu(x)  
        # layer 6
        x = self.lin4(x)
        x = F.softmax(x,dim=0)
        return x


In [79]:
model = Model()
model = model.to(device)
loss_fn = nn.CrossEntropyLoss()
learning_rate = 1e-4
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
fit(model,
    train_data=train_set,
    loss_fn=loss_fn,
    optimizer=optimizer,
    epochs=15,
    batch_size=64)

Epoch : 0 	Loss: 2.302584	 Accuracy:9.375%
Epoch : 0 	Loss: 2.209039	 Accuracy:48.356%
Epoch : 1 	Loss: 2.195250	 Accuracy:70.312%
Epoch : 1 	Loss: 2.191340	 Accuracy:64.970%
Epoch : 2 	Loss: 2.186258	 Accuracy:67.188%
Epoch : 2 	Loss: 2.191938	 Accuracy:67.241%
Epoch : 3 	Loss: 2.184922	 Accuracy:67.188%
Epoch : 3 	Loss: 2.192369	 Accuracy:68.865%
Epoch : 4 	Loss: 2.185551	 Accuracy:71.875%
Epoch : 4 	Loss: 2.190041	 Accuracy:69.985%
Epoch : 5 	Loss: 2.181469	 Accuracy:76.562%
Epoch : 5 	Loss: 2.187633	 Accuracy:70.924%
Epoch : 6 	Loss: 2.177785	 Accuracy:82.812%
Epoch : 6 	Loss: 2.186923	 Accuracy:71.912%
Epoch : 7 	Loss: 2.175957	 Accuracy:76.562%
Epoch : 7 	Loss: 2.184098	 Accuracy:72.811%
Epoch : 8 	Loss: 2.173222	 Accuracy:81.250%
Epoch : 8 	Loss: 2.183849	 Accuracy:73.843%
Epoch : 9 	Loss: 2.170268	 Accuracy:79.688%
Epoch : 9 	Loss: 2.184608	 Accuracy:74.301%
Epoch : 10 	Loss: 2.168926	 Accuracy:79.688%
Epoch : 10 	Loss: 2.184837	 Accuracy:74.582%
Epoch : 11 	Loss: 2.168279	 Acc

In [80]:
evaluate(model,batch_size=64,test_data=test_set)

Test accuracy:74.632% 
